<a href="https://colab.research.google.com/github/Adnan525/RAG/blob/main/RAG_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture
!pip install -q \
    llama-index \
    EbookLib \
    html2text \
    llama-index-embeddings-huggingface \
    llama-index-llms-ollama
!pip install llama-index-llms-huggingface

In [7]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [8]:
!mkdir -p "data/java/"

In [9]:
loader = SimpleDirectoryReader(
    input_dir="data/java/"
)

In [10]:
loader.input_files

[PosixPath('/content/data/java/JavaNotesForProfessionals.pdf'),
 PosixPath('/content/data/java/LearnFunctionalProgrammingTheFastWay-v1_1.pdf'),
 PosixPath('/content/data/java/ThinkingInJava-3rdEdition.pdf'),
 PosixPath('/content/data/java/Venkat Subramaniam - Functional Programming in Java (The Pragmatic Programmers) - 2014.pdf')]

In [11]:
documents = loader.load_data()

In [19]:
# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

# This will wrap the default prompts that are internal to llama-index
# taken from https://huggingface.co/Writer/camel-5b-hf
query_wrapper_prompt = PromptTemplate(
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [31]:
%%capture
!pip install bitsandbytes accelerate
!pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate bitsandbytes llama-index-readers-web

In [23]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

In [32]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}<|endoftext|>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}<|endoftext|>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}<|endoftext|>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n<|endoftext|>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt


llm = HuggingFaceLLM(
    model_name="stabilityai/stablelm-zephyr-3b",
    tokenizer_name="stabilityai/stablelm-zephyr-3b",
    query_wrapper_prompt=PromptTemplate("<|system|>\n<|endoftext|>\n<|user|>\n{query_str}<|endoftext|>\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.8},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
index = VectorStoreIndex.from_documents(documents)
from llama_index.core import StorageContext

persist_dir = "data/persistence_vector_store"
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
storage_context.persist(index)

In [ ]:
query_engine = index.as_query_engine()
query_engine.query("What is the difference between .map and .mapToObj?")